In [1]:
import numpy as np
import cartopy
from cartopy import crs
from cartopy.feature import NaturalEarthFeature
import matplotlib
import matplotlib.pyplot as plt
from netCDF4 import Dataset
import xarray as xr
import os

from wrf import (getvar, interplevel, vertcross, 
                 CoordPair, ALL_TIMES, to_np,
                 get_cartopy, latlon_coords,
                 cartopy_xlim, cartopy_ylim)

/home/lmar/.local/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/lmar/.local/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


The following code has been adapted from the WRF python tutorial presented by the National Centre for Atmospheric Research: https://github.com/NCAR/wrf_python_tutorial

### Importing WRF data

Here we access data from the forecast that was initialised at 1pm on the 14th May 2024, as can be infered from the directory name.

We choose the data valid at 13:00 on the first day of the forecast (2024-05-14) from domain 2.

The directory name - which implies the initialisation time - is a convention that we use to store and sort our data.

The filename - which implies the valid time - is the standard format for WRF output files.

Their generic format is:

`wrfout_d{domain number}_{YYYY}-{MM}-{DD}_{hh}:{mm}:{ss}`

We then create a `netCDF4` `Dataset` object of this wrfout.


In [15]:
WRF_DIRECTORY = "/data/shared_data/model/2024051412/" 
WRF_FILE = WRF_DIRECTORY + "wrfout_d03_2024-05-14_13:00:00"

wrfout = xr.open_dataset(WRF_FILE)

The `wrfout` variable represents a netCDF file, which we have extracted using `xarray`.

In a jupyter notebook `xarray` let's us take a look at our data nicely.

In [16]:
wrfout

<xarray.Dataset> Size: 234MB
Dimensions:                (Time: 1, south_north: 145, west_east: 320,
                            bottom_top: 50, bottom_top_stag: 51,
                            soil_layers_stag: 4, west_east_stag: 321,
                            south_north_stag: 146, seed_dim_stag: 8)
Coordinates:
    XLAT                   (Time, south_north, west_east) float32 186kB ...
    XLONG                  (Time, south_north, west_east) float32 186kB ...
    XTIME                  (Time) datetime64[ns] 8B ...
    XLAT_U                 (Time, south_north, west_east_stag) float32 186kB ...
    XLONG_U                (Time, south_north, west_east_stag) float32 186kB ...
    XLAT_V                 (Time, south_north_stag, west_east) float32 187kB ...
    XLONG_V                (Time, south_north_stag, west_east) float32 187kB ...
Dimensions without coordinates: Time, south_north, west_east, bottom_top,
                                bottom_top_stag, soil_layers_stag,
                                west_east_stag, south_north_stag, seed_dim_stag
Data variables: (12/165)
    Times                  (Time) |S19 19B ...
    LU_INDEX               (Time, south_north, west_east) float32 186kB ...
    ZNU                    (Time, bottom_top) float32 200B ...
    ZNW                    (Time, bottom_top_stag) float32 204B ...
    ZS                     (Time, soil_layers_stag) float32 16B ...
    DZS                    (Time, soil_layers_stag) float32 16B ...
    ...                     ...
    PCB                    (Time, south_north, west_east) float32 186kB ...
    PC                     (Time, south_north, west_east) float32 186kB ...
    LANDMASK               (Time, south_north, west_east) float32 186kB ...
    LAKEMASK               (Time, south_north, west_east) float32 186kB ...
    SST                    (Time, south_north, west_east) float32 186kB ...
    SST_INPUT              (Time, south_north, west_east) float32 186kB ...
Attributes: (12/134)
    TITLE:                            OUTPUT FROM WRF V4.4.2 MODEL
    START_DATE:                      2024-05-14_12:00:00
    SIMULATION_START_DATE:           2024-05-14_12:00:00
    WEST-EAST_GRID_DIMENSION:        321
    SOUTH-NORTH_GRID_DIMENSION:      146
    BOTTOM-TOP_GRID_DIMENSION:       51
    ...                              ...
    ISLAKE:                          21
    ISICE:                           15
    ISURBAN:                         13
    ISOILWATER:                      14
    HYBRID_OPT:                      2
    ETAC:                            0.2

We access data from the dataset as we would a python dictionary.

We can pull out something specific, such as 2m temperature.

Here we have use the variable name `'T2'` as our key: 

In [17]:
wrfout['XLAT'][:]

<xarray.DataArray 'XLAT' (Time: 1, south_north: 145, west_east: 320)> Size: 186kB
[46400 values with dtype=float32]
Coordinates:
    XLAT     (Time, south_north, west_east) float32 186kB ...
    XLONG    (Time, south_north, west_east) float32 186kB ...
    XTIME    (Time) datetime64[ns] 8B ...
Dimensions without coordinates: Time, south_north, west_east
Attributes:
    FieldType:    104
    MemoryOrder:  XY 
    description:  LATITUDE, SOUTH IS NEGATIVE
    units:        degree_north
    stagger:

We can access the data by using python list index notation:

In [18]:
wrfout['T2'][:,50,38]

<xarray.DataArray 'T2' (Time: 1)> Size: 4B
[1 values with dtype=float32]
Coordinates:
    XLAT     (Time) float32 4B ...
    XLONG    (Time) float32 4B ...
    XTIME    (Time) datetime64[ns] 8B ...
Dimensions without coordinates: Time
Attributes:
    FieldType:    104
    MemoryOrder:  XY 
    description:  TEMP at 2 M
    units:        K
    stagger:

### Extracting WRF variables using `getvar`

We will use the `wrf-python` function `getvar` to extract certain diagnostics from our wrfout files.

Some diagnostics come 'out of the box' in a wrfout, `getvar` extends this and also adds some nice functionality like concatenating the data from multiple wrfouts into a single file. 

The full list of diagnostics can be found here: https://wrf-python.readthedocs.io/en/latest/user_api/generated/wrf.getvar.html.

`getvar` is only compatible with `Dataset` object from the `netCDF4` package (even though it returns an `xarray` `DataArray` object...!?), so we will re read it:

In [19]:
wrfout = Dataset(WRF_FILE)

We will extract sea level pressure, using the variable name string `'slp'`:

In [20]:
slp = getvar(wrfout, 'slp')

In [21]:
slp

<xarray.DataArray 'slp' (south_north: 145, west_east: 320)> Size: 186kB
array([[ 998.6614 ,  998.6426 ,  998.67896, ..., 1001.5489 , 1001.59204,
        1001.5578 ],
       [ 998.6771 ,  998.67285,  998.66956, ..., 1001.56805, 1001.56726,
        1001.5562 ],
       [ 998.66345,  998.67346,  998.6809 , ..., 1001.57294, 1001.5864 ,
        1001.55695],
       ...,
       [ 999.0141 ,  999.03656,  999.0258 , ..., 1002.7905 , 1002.7832 ,
        1002.8167 ],
       [ 998.95776,  999.0402 ,  999.0296 , ..., 1002.7577 , 1002.7623 ,
        1002.82477],
       [ 998.95715,  999.0024 ,  999.0187 , ..., 1002.7969 , 1002.8057 ,
        1002.8139 ]], dtype=float32)
Coordinates:
    XLONG    (south_north, west_east) float32 186kB -3.986 -3.973 ... 0.01031
    XLAT     (south_north, west_east) float32 186kB 54.24 54.24 ... 55.28 55.28
    XTIME    float32 4B 60.0
    Time     datetime64[ns] 8B 2024-05-14T13:00:00
Dimensions without coordinates: south_north, west_east
Attributes:
    FieldType:    104
    MemoryOrder:  XY
    description:  sea level pressure
    units:        hPa
    stagger:      
    coordinates:  XLONG XLAT XTIME
    projection:   LambertConformal(stand_lon=-2.0, moad_cen_lat=57.0, truelat...

### Plotting with `cartopy`

`wrf-python` provides functions for retrieving spacial information from our `slp` object.

We will extract the projection and coordinates:

In [22]:
cart_proj = get_cartopy(slp)
lats, lons = latlon_coords(slp)

In [23]:
cart_proj

<Bound CRS: +proj=lcc +a=6370000.0 +b=6370000.0 +nadgrids=@nul ...>
Name: unknown
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unknown to WGS84
- method: NTv2
Datum: unknown
- Ellipsoid: unknown
- Prime Meridian: Greenwich
Source CRS: unknown

In [24]:
lats

<xarray.DataArray 'XLAT' (south_north: 145, west_east: 320)> Size: 186kB
array([[54.241455, 54.241665, 54.241875, ..., 54.242336, 54.242134,
        54.24193 ],
       [54.248676, 54.24889 , 54.2491  , ..., 54.249554, 54.249355,
        54.249146],
       [54.255898, 54.256107, 54.256313, ..., 54.256783, 54.256573,
        54.25637 ],
       ...,
       [55.267124, 55.267334, 55.267555, ..., 55.268032, 55.26782 ,
        55.267605],
       [55.274345, 55.274563, 55.274776, ..., 55.275253, 55.27504 ,
        55.27483 ],
       [55.28157 , 55.281788, 55.282005, ..., 55.282482, 55.28227 ,
        55.28205 ]], dtype=float32)
Coordinates:
    XLONG    (south_north, west_east) float32 186kB -3.986 -3.973 ... 0.01031
    XLAT     (south_north, west_east) float32 186kB 54.24 54.24 ... 55.28 55.28
    XTIME    float32 4B 60.0
    Time     datetime64[ns] 8B 2024-05-14T13:00:00
Dimensions without coordinates: south_north, west_east

In [25]:
lons

<xarray.DataArray 'XLONG' (south_north: 145, west_east: 320)> Size: 186kB
array([[-3.9858093e+00, -3.9734802e+00, -3.9611206e+00, ...,
        -6.6741943e-02, -5.4412842e-02, -4.2053223e-02],
       [-3.9861755e+00, -3.9738159e+00, -3.9614563e+00, ...,
        -6.6406250e-02, -5.4046631e-02, -4.1687012e-02],
       [-3.9865417e+00, -3.9741821e+00, -3.9618225e+00, ...,
        -6.6070557e-02, -5.3710938e-02, -4.1320801e-02],
       ...,
       [-4.0381775e+00, -4.0255127e+00, -4.0128174e+00, ...,
        -1.5777588e-02, -3.1127930e-03,  9.5825195e-03],
       [-4.0385742e+00, -4.0258789e+00, -4.0131836e+00, ...,
        -1.5411377e-02, -2.7160645e-03,  9.9487305e-03],
       [-4.0389404e+00, -4.0262451e+00, -4.0135803e+00, ...,
        -1.5045166e-02, -2.3498535e-03,  1.0314941e-02]], dtype=float32)
Coordinates:
    XLONG    (south_north, west_east) float32 186kB -3.986 -3.973 ... 0.01031
    XLAT     (south_north, west_east) float32 186kB 54.24 54.24 ... 55.28 55.28
    XTIME    float32 4B 60.0
    Time     datetime64[ns] 8B 2024-05-14T13:00:00
Dimensions without coordinates: south_north, west_east

We will now plot our data using matplotlib:

In [ ]:
# Create a figure and axes object
fig = plt.figure(figsize=(10,10))


# Set map projection with cart_proj
ax = plt.axes(projection=cart_proj)

# Add coastline features to axes object 
ax.coastlines(linewidth=0.5)

# Add features to the map like contour lines
# Set contour levels 
lvl = np.arange(980, 1030, 2.5) # Non-inclusive of end value (1030)

# Add contour lines
plt.contour(lons,
            lats,
            slp, 
            levels=lvl,
            colors='k',
            transform=crs.PlateCarree())

# Add filled contour lines
plt.contourf(lons,
             lats,
             slp,
             levels=lvl,
             transform=crs.PlateCarree(),
             cmap='viridis')

# Add colorbar
plt.colorbar(ax=ax, shrink=0.7, label='hPa')


# Manually set extent of map projection
ax.set_extent([-10, 2, 50, 59]) # Whole UK

### What's next?

Plot SLP for the other domains (1 and 3):
- Get the right file
- Change map extent
- Change contour levels if required

Try plotting another variable, look for what is available from wrf-python here: https://wrf-python.readthedocs.io/en/latest/user_api/generated/wrf.getvar.html